
Determine the main topics covered in a video with a machine learning model. It is best to extract tags or keywords from these videos. For example, for a video about politics and agriculture, two different topics are discussed. Our machine learning algorithm will have to be able to extract the words related to these two topics and it will have to be able in the end to say to which topic the video is the most strongly associated. In fact, the video may contain vocabulary words related to agriculture, but if the main subject is politics, the video will have to be classified in the political category with a higher probability.

We thus obtain after application of the model the words related to each topic (words that can serve as keys) and our video has been labeled.

It was also mentioned the possibility of doing sentiment analysis, to advise the user in the editing of his video. The detection of a video "fun"may allow to advise the user the use of certain theme, music or color.

Questions :

1- How to determine the different topics?
2- What is the maximum number of topics in a text that should be considered?
3- How to determine the maximum number of topics that our model must learn to recognize?


Comment déterminer les différentes topiques ?

In order to solve this problem, we proposed to use the Derichlet Latent Allocation algorithm. This algorithm for a given K of desired topics will analyze a corpus of texts and extract K topical. He will associate with each topic the words with the highest probability of belonging to the topic ((P (W | K)). It will be important to determine the good K.

The LDA output K set of words. It does not give the name of the topic associated with each set. How to draw this set of words? We will have to do it manually using a graphical interface.
 
Label the text with a single topic

We thought that it would be interesting to use the extracted keywords for each of the topics in order to lead a second model. This model will learn to associate a set of words with a single topic. At the output of the model, we obtain for K topical K probabilities. Thus, it will be possible to select N topics among the K found. Those with a probability greater than a threshold. 


In order to classify texts by topical 5 model were retained:




Scikit-learn:

* Linear-SVR
* KNN
Logistic regression
XGBoost

Deep-learning (with Pytorch) :

* Classification with attention based network (with BLSTM).


 
 Comparaison de model :
 https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568 
 
 Attention networks:
 https://humboldt-wi.github.io/blog/research/information_systems_1819/group5_han/
 https://towardsdatascience.com/attention-based-neural-machine-translation-b5d129742e2c
      

In [1]:
from sklearn.decomposition  import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 




In [2]:
from os import listdir
from os.path import isfile, join
import pandas as pd 
import pysrt

class LoadData:
    __df = pd.DataFrame()

    def __init__(self, path, extension='srt'):
        self.path = path 
        self.__files = [f for f in listdir(self.path) if (isfile(join(self.path, f)) and f.endswith(extension))]
    
    def files(self):
        return self.__files
    
    def data(self, verbose=True):
        textes = []
        for file in self.__files:
            if verbose :
                print(f"reading info of file {file}")
            texte = self.__treat_srt_file(file, verbose=verbose)
            textes.append(texte)
        self.__df['title'] = self.files()
        self.__df['texte'] = textes
        return self.__df
        
    def __treat_srt_file(self, file, verbose):
        f_path = "{}{}".format(self.path, file)
        print(f_path)
        f = pysrt.open(f_path, encoding='utf-8')
        article = ""
        for t in f:
            article += " " + t.text
        print(f"article {article}")
        return article
            
        
        

In [3]:
loader = LoadData(path='./TextFiles')

for file in loader.files():
    print(file)
    

supbiotect.srt
evo-comment-utiliser-easy-movie.srt
scor-simplifier-harmoniser.srt


In [4]:
df = loader.data()
df

reading info of file supbiotect.srt
./TextFilessupbiotect.srt


FileNotFoundError: [Errno 2] No such file or directory: './TextFilessupbiotect.srt'

CountVectorizer:

Convert a collection of text documents to a matrix of token counts


If you do not provide an a-priori dictionary and you do not use an analyzer that does some kind of feature 
selection then the number of features will be equal to the vocabulary size found by analyzing the data.

* max_df : If a word appear in more than 90% of the document it will be discarded 
* min_df : If a word appear less than 2 time it will not be take in count 

I used spacy french stop words list. We need to remove all the french 
stop word from our token because they appear in all the document and 
can be considered as topic's words. 

In [ ]:
import spacy
from spacy.lang.fr.examples import sentences 
import spacy.cli
spacy.cli.download("fr_core_news_sm")
spacy_nlp = spacy.load('fr_core_news_sm')

In [ ]:
spacy_stop_words = spacy.lang.fr.stop_words.STOP_WORDS

for i, w in enumerate(spacy_stop_words):
    print(w)
    if i >= 50:
        break 

In [ ]:
count_vectorizer = CountVectorizer(stop_words=spacy_stop_words)

In [ ]:
dtm = count_vectorizer.fit_transform(df['texte'])

LDA use the naives bayes probability to affect each word to a 
topic and text to a topic.

* each word is associated to a topic. This topic is the one that have the 
best chance to  generate the word in this document. Example the word 'formula one'
 have higher probabilty to be generated in a sport or car document that in a cooking one.
 
* The new theme is the one that have the best chance to generate the document.

* P(T | D) give the likelihood that the Doc D is affected to the Theme T  
* P(W | T) give the likelihood that the Word W is affected to the Theme T

Here we want to find the 10 topic. For each topic a list of word are choosen.

In [ ]:
LDA = LatentDirichletAllocation(n_components=10)

LDA.fit(dtm)


In [ ]:
for i, topic in enumerate(LDA.components_):
    print("\n\n\n\n")
    print(f"Topic n° {i}")
    last_ten_words = topic.argsort()[-5:]
    features = count_vectorizer.get_feature_names()
    for index in last_ten_words:
        print(features[index])

In [ ]:
supbiotech_vec = count_vectorizer.transform([df.iloc[0, 1]])

In [ ]:
saint_gobin_topics = LDA.transform(supbiotech_vec)

def extractTopic(components):
    last_ten_words = components.argsort()[-5:]
    features = count_vectorizer.get_feature_names()
    for index in last_ten_words:
        print(features[index])

In [ ]:
index = np.argmax(saint_gobin_topics)
extractTopic(LDA.components_[index])

We are going to add each labels array to his corresponding text.

In [ ]:
def extractTopicAndLoadInDataframe(count_vect, dataFrame, lda, nb_words=5):
    """
    
    :param cv: countVectorizer that was applied on the documents 
    :param df: the dataframe that contains the data 
    :param lda: Latent derichlet allocation trained 
    :param nb_words: number of words by topic that you want to used
    :return: 
    """
    document_labels = []
    for i, text in enumerate(dataFrame['texte']):
        text_vectorized = count_vect.transform([text])
        index = np.argmax(lda.transform(text_vectorized))
        topic = lda.components_[index]
        topic_word_index = topic.argsort()[-nb_words:] # get the 5 best word of the topic
        features = count_vect.get_feature_names()
        labels = []
        for j in topic_word_index:
            labels.append(features[j]) 
        document_labels.append(labels)
    dataFrame['labels'] = document_labels
        

In [ ]:
extractTopicAndLoadInDataframe(count_vect=count_vectorizer,dataFrame=df, lda=LDA, nb_words=5)

In [ ]:
df

I build a pipline. This pipline contains:
 * the cleaning of the data
   * lemming it's a normalization technique which converts high dimensional features into low dimensional features.
       We simply identify prefix and suffix and produce the simplest version of the word. 
       For example : connect, connection, connected => have different grammatical function but for ML models we need
       that this words be one simple word (connect) because it's can be confusing.
       
   * tokenizing : transform into token 
   * removing punct : It's useless for text classification 
   * removing stop words : stop words can lead to poor performance. 
  
 * Vectorize the data with CountVectorizer: Convert a collection of text documents to a matrix of token counts
 * Apply the LDA on the data : have been previously explained. 

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

import spacy.cli
from spacy.lang.fr import French
spacy.cli.download("fr_core_news_sm")
spacy_nlp = spacy.load('fr_core_news_sm')

In [ ]:
parser = French()

tokenizer = French()

stop_words = spacy.lang.fr.stop_words.STOP_WORDS

punctuations = spacy.lang.fr.punctuation.LIST_PUNCT

def easy_tokenizer(sentence):
    """
    
    :param sentence: the texte we want to treat with our parser 
    :return: 
    """
    # creating our token object 
    easy_tokens = parser(sentence)
    
    # lemming  and lowering. Stripping white space 
    # if the word is not a Pronoun lower the word and strip space else return the lower word. 
    # Because  spacy replace pronouns by "-PRON-" on lemming call. as we need the pronoun we can not do its lemmatization
    
    easy_tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in easy_tokens ]
    
    
    # remove stop words 
    easy_tokens = [word  for word in easy_tokens if (word not in stop_words and word not in punctuations)]

    
    return easy_tokens

In [ ]:
# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

class PrepareText(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X):
        """
        Take a dataframe as paremeter and fit to the data set è
        :param X: 
        :return: 
        """
        return self
    
    def transform(self, X):
        """
        remove stop word in the text and retore the text
        return a data frame that contains all the text 
        :param X: 
        :return: 
        """
        return [clean_text(text) for text in X]
    

In [ ]:
easy_vectorizer = CountVectorizer(tokenizer=easy_tokenizer, ngram_range=(1,1))

In [ ]:


from sklearn.pipeline import Pipeline
easy_lda = LatentDirichletAllocation(n_components=5)
X = df['texte']
easy_pipeline = Pipeline([('vectorizer', easy_vectorizer),("topic-analyser", easy_lda)])
easy_pipeline.fit(X=X)
topic_analysis = easy_pipeline.transform(X=X)

In [ ]:
extractTopicAndLoadInDataframe(count_vect=easy_vectorizer,dataFrame=df, lda=easy_lda, nb_words=5)

In [ ]:
df
